In [2]:
import pandas as pd
import numpy as np
import datetime
import nbformat

import seaborn as sns
pal = sns.color_palette()

import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly import tools
import plotly.io as pio
import io
pio.renderers.default = "vscode"

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')

from sklearn.preprocessing import  StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LogisticRegression
from IPython.display import display
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # to avoid deprecation warnings

In [2]:

# # lecture du fichier Excel
# converted = pd.read_excel("./assets/get_around_delay_analysis.xlsx")

# # écriture du fichier CSV
# converted.to_csv("./assets/get_around_delay_analysis.csv", index=False)


 # FILES READING

In [3]:
# Import dataset
print("Loading dataset...")
df_pricing = pd.read_csv("./assets/get_around_pricing_project.csv")  # sep = ";"
print("...Done.")
df_delay = pd.read_csv("./assets/get_around_delay_analysis.csv")
print("...Done.")
pd.set_option('display.max_columns', None)

Loading dataset...
...Done.
...Done.


In [11]:
# Basic stats
print('Basics informations for df_pricing')
print()
print("Number of rows : {}".format(df_pricing.shape[0]))
print("Number of columns : {}".format(df_pricing.shape[1]))
print()

print("Display of dataset: ")
display(df_pricing.head(1))
print()

print("Basics statistics: ")
data_desc = df_pricing.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")

pd.concat([df_pricing.isnull().sum(), 100 * df_pricing.isnull().sum()/len(df_pricing)], axis=1).rename(columns={0:'Missing Records', 1:'Percentage (%)'})


Basics informations for df_pricing

Number of rows : 4843
Number of columns : 15

Display of dataset: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106



Basics statistics: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843.000000,4843,4.843000e+03,4843.00000,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.000000
unique,NaN,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,NaN,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,NaN,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,2421.000000,NaN,1.409628e+05,128.98823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.214536
std,1398.198007,NaN,6.019674e+04,38.99336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.568268
min,0.000000,NaN,-6.400000e+01,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.000000
25%,1210.500000,NaN,1.029135e+05,100.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.000000
50%,2421.000000,NaN,1.410800e+05,120.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.000000
75%,3631.500000,NaN,1.751955e+05,135.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.000000



Percentage of missing values: 


,Missing Records,Percentage (%)
Unnamed: 0,0,0.0
model_key,0,0.0
mileage,0,0.0
engine_power,0,0.0
fuel,0,0.0
paint_color,0,0.0
car_type,0,0.0
private_parking_available,0,0.0
has_gps,0,0.0
has_air_conditioning,0,0.0
